In [1]:
%matplotlib inline
import os, sys
sys.path.append('/Users/hhsieh/anaconda3/envs/astroconda/lib/python3.6/site-packages')
sys.path.append('/opt/local/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages')
print(sys.version)
from astropy.io import fits
from astropy.io.fits import getheader
from astropy.modeling import models
from astropy import nddata
from astropy import units as u
import astropy.units as u
import astropy.coordinates as coord
from astropy.time import Time, TimeDelta
from astropy.table import Table
from astropy.coordinates import SkyCoord,Angle
from astroquery.jplhorizons import Horizons
import ccdproc as cp
from ccdproc import CCDData, ImageFileCollection
import numpy as np
import math, glob, os, bz2, subprocess, sys
import cosmics_py3
import scipy.signal as signal
import scipy.ndimage as ndimage
import datetime


3.6.11 | packaged by conda-forge | (default, Aug  5 2020, 20:19:23) 
[GCC Clang 10.0.1 ]


In [2]:
def compress_file_fpack(filename):
    fpack_command  = '/Users/hhsieh/Astro/tools/cfitsio/fpack'
    delete_command = '/bin/rm'
    fpack_cmd = [fpack_command,filename]
    process = subprocess.call(fpack_cmd)
    if process == 0:
        delete_cmd = [delete_command,filename]
        process = subprocess.call(delete_cmd)
    else:
        print('fpack command failed.')
    return None

def decompress_file_fpack(filename):
    funpack_command = '/Users/hhsieh/Astro/tools/cfitsio/funpack'
    delete_command  = '/bin/rm'
    funpack_cmd = [funpack_command,filename]
    process = subprocess.call(funpack_cmd)
    if process == 0:
        delete_cmd = [delete_command,filename]
        process = subprocess.call(delete_cmd)
    else:
        print('funpack command failed.')
    return None

def convert_to_long_target_desig(short_desig):
    long_desig = short_desig
    if short_desig == '133P':        long_desig = '133P/1996 N2 (Elst-Pizarro 1)'
    if short_desig == '176P':        long_desig = '176P/1999 RE70 (LINEAR 52)'
    if short_desig == '233P':        long_desig = '233P/2005 JR71 (La Sagra)'
    if short_desig == '238P':        long_desig = '238P/2005 U1 (Read 3)'
    if short_desig == '259P':        long_desig = '259P/2008 R1 (Garradd 4)'
    if short_desig == '2006 VW139':  long_desig = '288P/2006 VW139 (300163)'
    if short_desig == '288P':        long_desig = '288P/2006 VW139 (300163)'
    if short_desig == '313P':        long_desig = '313P/2014 S4 (Gibbs 16)'
    if short_desig == 'P/2010 R2':   long_desig = '324P/2010 R2 (La Sagra 4)'
    if short_desig == '324P':        long_desig = '324P/2010 R2 (La Sagra 4)'
    if short_desig == '358P':        long_desig = '358P/2012 T1 (PanSTARRS 11)'
    if short_desig == 'P/2012 T1':   long_desig = '358P/2012 T1 (PanSTARRS 11)'
    if short_desig == 'P/2013 P5':   long_desig = '311P/2013 P5 (PanSTARRS)'
    if short_desig == '2013 P5':     long_desig = '311P/2013 P5 (PanSTARRS)'
    if short_desig == '197P':        long_desig = '197P/2003 KV2 (LINEAR 30)'
    if short_desig == 'P/2016 J1-A': long_desig = 'P/2016 J1-A (PANSTARRS)'
    if short_desig == 'P/2015 X6':   long_desig = 'P/2015 X6 (PANSTARRS)'
    return long_desig

#def convert_to_short_target_desig(long_desig):
#    short_desig = long_desig
#    if long_desig == '133P/1996 N2 (Elst-Pizarro 1)': short_desig = '133P'
#    if long_desig == '176P/1999 RE70 (LINEAR 52)':    short_desig = '176P'
#    if long_desig == '238P/2005 U1 (Read 3)':         short_desig = '238P'
#    if long_desig == '259P/2008 R1 (Garradd 4)':      short_desig = '259P'
#    if long_desig == '288P/2006 VW139 (300163)':      short_desig = '288P'
#    if long_desig == '313P/2014 S4 (Gibbs 16)':       short_desig = '313P'
#    if long_desig == '324P/2010 R2 (La Sagra 4)':     short_desig = '324P'
#    if long_desig == '358P/2012 T1 (PanSTARRS 11)':   short_desig = '358P'
#    return long_desig


#def query_horizons(object_id,obs_code,obs_datetime,time_step):
#    # Create and execute JPL Horizons query for a given object, observatory code, date, and time step
#    date_start = Time('{:s}'.format(obs_datetime),scale='utc',format='iso')
#    dt = TimeDelta(1,format='jd')
#    date_end = date_start+1
#    obj = Horizons(id=object_id, location=obs_code,epochs={'start':date_start.iso,'stop':date_end.iso,'step':time_step})
#    ephems = obj.ephemerides()
#    return ephems

def get_geometry_params(object_name,obs_date,obs_time,exp_time):
    obs_code  = '500'   # observatory code
    time_step = '2d'
    date_start = Time('{:s} {:s}'.format(obs_date,obs_time),scale='utc',format='iso')
    mid_dt = TimeDelta(exp_time,format='sec') / 2
    date_mid = date_start + mid_dt
    #date_next = date_mid + 1
    date_next = date_start + 1
    #print(date_mid.jd)
    #obj = Horizons(id=object_name,location=obs_code,epochs={'start':date_mid.iso,'stop':date_next.iso,'step':time_step})
    obj = Horizons(id=object_name,location=obs_code,epochs=date_mid.jd)
    #print(object_name,obs_code,date_mid.iso,date_next.iso,time_step)
    #print(object_name,obs_code,date_mid.jd)
    #print(obj)
    ephems = obj.ephemerides()
    #print(ephems.columns)
    ra = ephems[0]['RA']
    dec = ephems[0]['DEC']
    ra_rate = ephems[0]['RA_rate']
    dec_rate = ephems[0]['DEC_rate']
    heliodist = ephems[0]['r']
    geodist   = ephems[0]['delta']
    phsang    = ephems[0]['alpha']
    antisolarPA = ephems[0]['sunTargetPA']
    neghelioPA  = ephems[0]['velocityPA']
    trueanom    = ephems[0]['true_anom']
    ra_angle = Angle(ra * u.deg)
    dec_angle = Angle(dec * u.deg)
    ra_hms = ra_angle.to_string(unit=u.hour,sep=':',precision=1,pad=True)
    dec_dms = dec_angle.to_string(unit=u.degree,sep=':',precision=0,pad=True)
    
    return ra_hms,dec_dms,ra_rate,dec_rate,heliodist,geodist,phsang,antisolarPA,neghelioPA,trueanom
    

#def retrieve_nightly_ephems(object_id,obs_code,obs_datetime,time_step_min):
#    # Sends a query to JPL Horizons for a given object, observatory code, date, and time step,
#    #   and outputs date/time, RA, and Dec to an astropy Table
#    time_step = '{:d}m'.format(time_step_min)
#    ephems = query_horizons(object_id,obs_code,obs_datetime,time_step)
#    num_lines_all = len(ephems)
#    num_lines     = 0
#    
#    # Count ephemeris entries that meet night-time (or astronomical twilight) and airmass requirements
#    for idx in range(0,num_lines_all):
#        if (ephems[idx]['solar_presence'] == '' or ephems[idx]['solar_presence'] == 'A') and ephems[idx]['airmass'] < 2.5:
#            num_lines += 1
#    
#    # Initialize ephemeris arrays
#    eph_datetime_str = [0 for idx in range(num_lines)]
#    eph_ra           = [0 for idx in range(num_lines)]
#    eph_dec          = [0 for idx in range(num_lines)]
#
#    # Save ephemeris entries that meet night-time (or astronomical twilight) and airmass requirements
#    idx2 = 0
#    for idx1 in range(0,num_lines_all):
#        if (ephems[idx1]['solar_presence'] == '' or ephems[idx1]['solar_presence'] == 'A') and ephems[idx1]['airmass'] < 2.5:
#            eph_datetime_str[idx2] = ephems[idx1]['datetime_str']
#            eph_ra[idx2]           = ephems[idx1]['RA']
#            eph_dec[idx2]          = ephems[idx1]['DEC']
#            idx2 += 1
#
#    # Create ephemeris table for entries that meet night-time (or astronomical twilight) and airmass requirements
#    ephem_table = Table()
#    ephem_table = Table([eph_datetime_str,eph_ra,eph_dec],names=('datetime','ra','dec'),dtype=('S17','f8','f8'))
#    ephem_table['clear'] = True
#
#    # Confirm start and end times and positions of object when it is observable
#    print('Start time and position: {:s} {:f} {:f}'.format(ephem_table[0]['datetime'],ephem_table[0]['ra'],ephem_table[0]['dec']))
#    print('End time and position:   {:s} {:f} {:f}'.format(ephem_table[len(ephem_table)-1]['datetime'],ephem_table[len(ephem_table)-1]['ra'],ephem_table[len(ephem_table)-1]['dec']))
#    
#    return ephem_table

def update_metadata(fits_file):
    with fits.open(fits_file) as hdulist:
        data = hdulist[0].data
        hdr = hdulist[0].header
    
    # Insert full object designation header field
    object_name = hdr['OBJECT']
    object_desig = convert_to_long_target_desig(object_name)
    hdr['OBJDESIG'] = (object_desig,'Full object designation')

    # Shift azimuth from -180-to-180 scale to 0-to-360 scale
    azimuth = hdr['AZIMUTH']
    if azimuth < 0: hdr['AZIMUTH'] = azimuth + 360
    
    # Add additional keywords if not present
    if 'ORIGIN'   not in hdr: hdr['ORIGIN']   = ('N/A','FITS file originator')
    if 'IRAF-TLM' not in hdr: hdr['IRAF-TLM'] = ('N/A','Time of last modification')
    
    # Ensure all images have both OIWFS and PWFS2 guider information
    if 'OIARA'    not in hdr: hdr['OIARA']    = ('N/A','RA of OIWFS guide star')
    if 'OIARV'    not in hdr: hdr['OIARV']    = ('N/A','OIWFS Heliocentric Radial Velocity')
    if 'OIAWAVEL' not in hdr: hdr['OIAWAVEL'] = ('N/A','OIWFS Effective Target Wavelength')
    if 'OIADEC'   not in hdr: hdr['OIADEC']   = ('N/A','Declination of OIWFS guide star')
    if 'OIAEPOCH' not in hdr: hdr['OIAEPOCH'] = ('N/A','Epoch for OIWFS guide star coordinates')
    if 'OIAEQUIN' not in hdr: hdr['OIAEQUIN'] = ('N/A','Equinox for OIWFS guide star coordinates')
    if 'OIAFRAME' not in hdr: hdr['OIAFRAME'] = ('N/A','OIWFS Target co-ordinate system')
    if 'OIAOBJEC' not in hdr: hdr['OIAOBJEC'] = ('N/A','Object Name for OIWFS, Chop A')
    if 'OIAPMDEC' not in hdr: hdr['OIAPMDEC'] = ('N/A','OIWFS Proper Motion in Declination')
    if 'OIAPMRA'  not in hdr: hdr['OIAPMRA']  = ('N/A','OIWFS Proper Motion in RA')
    if 'OIAPARAL' not in hdr: hdr['OIAPARAL'] = ('N/A','OIWFS Parallax of Target')
    if 'OIFREQ'   not in hdr: hdr['OIFREQ']   = ('N/A','OIWFS sampling frequency (Hz)')

    if 'P2ARA'    not in hdr: hdr['P2ARA']    = ('N/A','RA of PWFS2 guide star')
    if 'P2ARV'    not in hdr: hdr['P2ARV']    = ('N/A','PWFS2 Heliocentric Radial Velocity')
    if 'P2AWAVEL' not in hdr: hdr['P2AWAVEL'] = ('N/A','PWFS2 Effective Target Wavelength')
    if 'P2ADEC'   not in hdr: hdr['P2ADEC']   = ('N/A','Declination of PWFS2 guide star')
    if 'P2AEPOCH' not in hdr: hdr['P2AEPOCH'] = ('N/A','Epoch for PWFS2 guide star coordinates')
    if 'P2AEQUIN' not in hdr: hdr['P2AEQUIN'] = ('N/A','Equinox for PWFS2 guide star coordinates')
    if 'P2AFRAME' not in hdr: hdr['P2AFRAME'] = ('N/A','PWFS2 Target co-ordinate system')
    if 'P2AOBJEC' not in hdr: hdr['P2AOBJEC'] = ('N/A','Object Name for PWFS 2, Chop A')
    if 'P2APMDEC' not in hdr: hdr['P2APMDEC'] = ('N/A','PWFS2 Proper Motion in Declination')
    if 'P2APMRA'  not in hdr: hdr['P2APMRA']  = ('N/A','PWFS2 Proper Motion in RA')
    if 'P2APARAL' not in hdr: hdr['P2APARAL'] = ('N/A','PWFS2 Parallax of Target')
    if 'P2FOCUS'  not in hdr: hdr['P2FOCUS']  = ('N/A','PWFS2 Focus Offset (mm)')
    if 'P2FREQ'   not in hdr: hdr['P2FREQ']   = ('N/A','PWFS2 sampling frequency (Hz)')
    
    if 'P1ARA'    not in hdr: hdr['P1ARA']    = ('N/A','RA of PWFS1 guide star')
    if 'P1ARV'    not in hdr: hdr['P1ARV']    = ('N/A','PWFS1 Heliocentric Radial Velocity')
    if 'P1AWAVEL' not in hdr: hdr['P1AWAVEL'] = ('N/A','PWFS1 Effective Target Wavelength')
    if 'P1ADEC'   not in hdr: hdr['P1ADEC']   = ('N/A','Declination of PWFS1 guide star')
    if 'P1AEPOCH' not in hdr: hdr['P1AEPOCH'] = ('N/A','Epoch for PWFS1 guide star coordinates')
    if 'P1AEQUIN' not in hdr: hdr['P1AEQUIN'] = ('N/A','Equinox for PWFS1 guide star coordinates')
    if 'P1AFRAME' not in hdr: hdr['P1AFRAME'] = ('N/A','PWFS1 Target co-ordinate system')
    if 'P1AOBJEC' not in hdr: hdr['P1AOBJEC'] = ('N/A','Object Name for PWFS 1, Chop A')
    if 'P1APMDEC' not in hdr: hdr['P1APMDEC'] = ('N/A','PWFS1 Proper Motion in Declination')
    if 'P1APMRA'  not in hdr: hdr['P1APMRA']  = ('N/A','PWFS1 Proper Motion in RA')
    if 'P1APARAL' not in hdr: hdr['P1APARAL'] = ('N/A','PWFS1 Parallax of Target')
    if 'P1FOCUS'  not in hdr: hdr['P1FOCUS']  = ('N/A','PWFS1 Focus Offset (mm)')
    if 'P1FREQ'   not in hdr: hdr['P1FREQ']   = ('N/A','PWFS1 sampling frequency (Hz)')

    # Ensure all images have pause-related header keywords
    if 'EXPPAU00' not in hdr: hdr['EXPPAU00'] = ('N/A','UT pause')
    if 'EXPRES00' not in hdr: hdr['EXPRES00'] = ('N/A','UT of continue')

    # Ensure all images contain maximum number of bias file slots
    if 'BIAS_2' not in hdr: hdr['BIAS_2'] = ('N/A','2nd bias frame used')
    if 'BIAS_3' not in hdr: hdr['BIAS_3'] = ('N/A','3rd bias frame used')
    if 'BIAS_4' not in hdr: hdr['BIAS_4'] = ('N/A','4th bias frame used')
    
    if hdr['BIAS_2'] == 'n/a': hdr['BIAS_2'] = 'N/A'
    if hdr['BIAS_3'] == 'n/a': hdr['BIAS_3'] = 'N/A'
    if hdr['BIAS_4'] == 'n/a': hdr['BIAS_4'] = 'N/A'

    # Add requested timing window keywords if not present
    if 'REQTWS01' not in hdr: hdr['REQTWS01'] = ('N/A','Requested Timing Window 01 Start (UTC)')
    if 'REQTWD01' not in hdr: hdr['REQTWD01'] = ('N/A','Requested Timing Window 01 Duration [seconds]')
    if 'REQTWN01' not in hdr: hdr['REQTWN01'] = ('N/A','Requested Timing Window 01 Number of Repeats')
    if 'REQTWP01' not in hdr: hdr['REQTWP01'] = ('N/A','Requested Timing Window 01 Repeat Period [secon')
    
    # Update comments on reduction status keywords
    hdr['OSCANSUB'] = (hdr['OSCANSUB'],'Overscan subtraction performed? (T/F)')
    hdr['TRIMMED']  = (hdr['TRIMMED'],'Image trimming performed? (T/F)')
    hdr['ZEROCORR'] = (hdr['ZEROCORR'],'Bias subtraction performed? (T/F)')
    hdr['FLATCORR'] = (hdr['FLATCORR'],'Flatfield correction performed? (T/F)')
    hdr['CRCORR']   = (True,'Cosmic ray removal performed? (T/F)')
    hdr['RADESYSA'] = (hdr['RADESYSA'],'Equatorial coordinate system')
    hdr['BUNIT']    = (hdr['BUNIT'],'Unit of original pixel value')
    
    # Remove checksums
    if 'CHECKSUM' in hdr: del hdr['CHECKSUM']
    if 'DATASUM'  in hdr: del hdr['DATASUM']
        
    fits.writeto(fits_file,data,hdr,overwrite=True,checksum=True)
    
    return None


def get_metadata(fits_file):
    object_name,object_desig,obs_date,obs_time,exp_time,filter_name,obs_class = '','','','','','',''

    with fits.open(fits_file) as hdulist:
        data = hdulist[0].data
        hdr = hdulist[0].header
        
    object_name = hdr['OBJECT']
    obs_date    = hdr['DATE']
    obs_time    = hdr['UTSTART'][:10]
    exp_time    = hdr['EXPTIME']
    filter_name = hdr['FILTER2'][:1]
    object_desig = hdr['OBJDESIG']
    obs_class = hdr['OBSCLASS'].strip()
    #date_start  = Time('{:s} {:s}'.format(obs_date,utstart),scale='utc',format='iso')
    #obs_time    = date_start.iso[:10]
    #print(obs_time)
    
    #object_desig = convert_to_long_target_desig(object_name)
    #hdr['OBJDESIG'] = (object_desig,'Full object designation')
    #exp_time = int(round(elapsed))
    #hdr['EXPTIME'] = exp_time
    
    return object_name,object_desig,obs_date,obs_time,exp_time,filter_name,obs_class

print('Done.')


Done.


In [3]:
def get_update_object_info(base_dir):
    print('{:s} - Starting processing data in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),base_dir))
    dir_procfits = 'flatfits_pyt'
    
    #if base_dir[-6:-1] == 'LP-11':  output_file  = base_dir + 'target_table_' + base_dir[-15:-1] + '.txt'
    #if base_dir[-7:-1] == 'LP-104': output_file  = base_dir + 'target_table_' + base_dir[-16:-1] + '.txt'
    
    if base_dir[-28:-17] == 'gemini_data': output_file = base_dir + 'target_table_' + base_dir[-16:-1] + '.txt' # for Q-64
    if base_dir[-27:-16] == 'gemini_data': output_file = base_dir + 'target_table_' + base_dir[-15:-1] + '.txt' # for LP-11
    if base_dir[-26:-15] == 'gemini_data': output_file = base_dir + 'target_table_' + base_dir[-14:-1] + '.txt' # for Q-64
    
    #output_file  = base_dir + 'target_table_' + base_dir[-16:-1] + '.txt' # for LP-104
    #output_file  = base_dir + 'target_table_' + base_dir[-15:-1] + '.txt' # for LP-11
    #output_file  = base_dir + 'target_table_' + base_dir[-14:-1] + '.txt' # for Q-64

    with open(output_file,'w') as of:
        #of.write('File Name      Telescope    Target       LongDesig                        ObsDate    ObsStartTime    ExpTime  Filter     RA          Dec     RA_rate  Dec_rate    r      delta   phsAng  antiSolarPA  helioVelPA  trueAnom\n')
        of.write('File Name      Target       LongDesig                        ObsDate    ObsStartTime    ExpTime  Filter     RA          Dec     RA_rate  Dec_rate    r      delta   phsAng  antiSolarPA  helioVelPA  trueAnom\n')

    os.chdir(base_dir)
    for data_dir in sorted(glob.glob('ut*_gemini?')):
        path_procfits = base_dir + data_dir + '/' + dir_procfits + '/'
    
        if data_dir[-1:] == 'N':   instr_id = 'gmosn'
        elif data_dir[-1:] == 'S': instr_id = 'gmoss'
        else: instr_id = 'not recognized'

        if instr_id == 'gmosn' or instr_id == 'gmoss':
            if instr_id == 'gmosn': telescope = 'Gemini-N'
            if instr_id == 'gmoss': telescope = 'Gemini-S'
            if os.path.exists(path_procfits):
                os.chdir(path_procfits)
                for fz_file in sorted(glob.glob('*.chip2.otzfc.fits.fz')):
                    print('{:s} - Processing {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),fz_file[:-3]))

                    # Decompress fpacked FITS files
                    decompress_file_fpack(fz_file)
                    decompress_file_fpack(fz_file[:-15] + '1.otzfc.fits.fz')
                    decompress_file_fpack(fz_file[:-15] + '3.otzfc.fits.fz')
                
                    # Update image headers to ensure PDS compliance
                    fits_file = fz_file[:-3]
                    update_metadata(fits_file)
                    update_metadata(fits_file[:-12] + '1.otzfc.fits')
                    update_metadata(fits_file[:-12] + '3.otzfc.fits')
                
                    # Retrieve image metadata from headers of chip2 files
                    object_name,object_desig,obs_date,obs_time,exp_time,filter_name,obs_class = get_metadata(fits_file)
                
                    # Recompress FITS files
                    compress_file_fpack(fits_file)
                    compress_file_fpack(fits_file[:-12] + '1.otzfc.fits')
                    compress_file_fpack(fits_file[:-12] + '3.otzfc.fits')
                
                    # Retrieve geometry data for objects in images
                    if obs_class == 'science':
                        ra_hms,dec_dms,ra_rate,dec_rate,heliodist,geodist,phsang,antisolarPA,neghelioPA,trueanom = get_geometry_params(object_name,obs_date,obs_time,exp_time)
                
                        # Write metadata and geometry data to output file
                        with open(output_file,'a') as of:
                            #of.write('{:s}  {:<10s}   {:<11s}  {:<30s}  {:s}   {:s}       {:>4d}     {:s}     {:>8s}  {:>9s}   {:6.1f}   {:6.1f}  {:7.4f}  {:7.4f}   {:4.1f}      {:5.1f}        {:5.1f}      {:5.1f}\n'.format(fits_file[:13],telescope,object_name,object_desig,obs_date,obs_time,exp_time,filter_name,ra_hms,dec_dms,ra_rate,dec_rate,heliodist,geodist,phsang,antisolarPA,neghelioPA,trueanom))
                            of.write('{:s}  {:<11s}  {:<30s}  {:s}   {:s}       {:>4d}     {:s}     {:>8s}  {:>9s}   {:6.1f}   {:6.1f}  {:7.4f}  {:7.4f}   {:4.1f}      {:5.1f}        {:5.1f}      {:5.1f}\n'.format(fits_file[:13],object_name,object_desig,obs_date,obs_time,int(exp_time),filter_name,ra_hms,dec_dms,ra_rate,dec_rate,heliodist,geodist,phsang,antisolarPA,neghelioPA,trueanom))
            else:
                print('{:s} - Processed science data directory in {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
        else:
            print('{:s} - Directory name format not recognized for {:s}.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
    
    print('{:s} - Finished processing data in {:s}.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),base_dir))
    
    return None
    
def get_object_info(base_dir):
    print('{:s} - Starting processing data in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),base_dir))
    dir_procfits = 'flatfits_pyt'
    
    #if base_dir[-6:-1] == 'LP-11':  output_file  = base_dir + 'target_table_' + base_dir[-15:-1] + '.txt'
    #if base_dir[-7:-1] == 'LP-104': output_file  = base_dir + 'target_table_' + base_dir[-16:-1] + '.txt'
    
    if base_dir[-28:-17] == 'gemini_data': output_file = base_dir + 'target_table_' + base_dir[-16:-1] + '.txt' # for Q-64
    if base_dir[-27:-16] == 'gemini_data': output_file = base_dir + 'target_table_' + base_dir[-15:-1] + '.txt' # for LP-11
    if base_dir[-26:-15] == 'gemini_data': output_file = base_dir + 'target_table_' + base_dir[-14:-1] + '.txt' # for Q-64
    
    #output_file  = base_dir + 'target_table_' + base_dir[-16:-1] + '.txt' # for LP-104
    #output_file  = base_dir + 'target_table_' + base_dir[-15:-1] + '.txt' # for LP-11
    #output_file  = base_dir + 'target_table_' + base_dir[-14:-1] + '.txt' # for Q-64

    with open(output_file,'w') as of:
        of.write('File Name      Telescope    Target       LongDesig                        ObsDate    ObsStartTime    ExpTime  Filter     RA          Dec     RA_rate  Dec_rate    r      delta   phsAng  antiSolarPA  helioVelPA  trueAnom\n')

    os.chdir(base_dir)
    for data_dir in sorted(glob.glob('ut*_gemini?')):
        path_procfits = base_dir + data_dir + '/' + dir_procfits + '/'
    
        if data_dir[-1:] == 'N':   instr_id = 'gmosn'
        elif data_dir[-1:] == 'S': instr_id = 'gmoss'
        else: instr_id = 'not recognized'

        if instr_id == 'gmosn' or instr_id == 'gmoss':
            if instr_id == 'gmosn': telescope = 'Gemini-N'
            if instr_id == 'gmoss': telescope = 'Gemini-S'
            if os.path.exists(path_procfits):
                os.chdir(path_procfits)
                for fz_file in sorted(glob.glob('*.chip2.otzfc.fits.fz')):
                    print('{:s} - Processing {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),fz_file[:-3]))

                    # Decompress fpacked FITS files
                    decompress_file_fpack(fz_file)
                    fits_file = fz_file[:-3]
                
                    # Retrieve image metadata from headers of chip2 files
                    object_name,object_desig,obs_date,obs_time,exp_time,filter_name,obs_class = get_metadata(fits_file)
                
                    # Recompress FITS files
                    compress_file_fpack(fits_file)
                
                    # Retrieve geometry data for objects in images
                    if obs_class == 'science':
                        ra_hms,dec_dms,ra_rate,dec_rate,heliodist,geodist,phsang,antisolarPA,neghelioPA,trueanom = get_geometry_params(object_name,obs_date,obs_time,exp_time)
                
                        # Write metadata and geometry data to output file
                        with open(output_file,'a') as of:
                            of.write('{:s}  {:<10s}   {:<11s}  {:<30s}  {:s}   {:s}       {:>4d}     {:s}     {:>8s}  {:>9s}   {:6.1f}   {:6.1f}  {:7.4f}  {:7.4f}   {:4.1f}      {:5.1f}        {:5.1f}      {:5.1f}\n'.format(fits_file[:13],telescope,object_name,object_desig,obs_date,obs_time,int(exp_time),filter_name,ra_hms,dec_dms,ra_rate,dec_rate,heliodist,geodist,phsang,antisolarPA,neghelioPA,trueanom))
            else:
                print('{:s} - Processed science data directory in {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
        else:
            print('{:s} - Directory name format not recognized for {:s}.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
    
    print('{:s} - Finished processing data in {:s}.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),base_dir))
    
    return None


In [4]:
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2016B-LP-11/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2017A-LP-11/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2017B-LP-11/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2018A-LP-11/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2018B-LP-11/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2019A-LP-11/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2016B-LP-11/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2017A-LP-11/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2017B-LP-11/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2018A-LP-11/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2018B-LP-11/')
#get_update_object_info('/volumes/Fantom8a/BackupData/gemini/LLP_data/gemini_data.GS-2019B-LP-104/')
#get_update_object_info('/volumes/Fantom8a/BackupData/gemini/data_queue_programs/gemini_data.GN-2016A-Q-88/')
#get_update_object_info('/volumes/Fantom8a/BackupData/gemini/data_queue_programs/gemini_data.GS-2015B-Q-64/')
#get_update_object_info('/volumes/Fantom8a/BackupData/gemini/data_queue_programs/gemini_data.GN-2013A-Q-102/')
#get_update_object_info('/volumes/Fantom8a/BackupData/gemini/data_queue_programs/gemini_data.GN-2014B-Q-91/')
#get_update_object_info('/volumes/Fantom8a/BackupData/gemini/data_queue_programs/gemini_data.GN-2016A-Q-88/')
#get_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2016B-LP-11/')
#get_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2017A-LP-11/')
#get_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2017B-LP-11/')
#get_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2018A-LP-11/')
#get_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2018B-LP-11/')
#get_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2019A-LP-11/')
#get_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2016B-LP-11/')
#get_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2017A-LP-11/')
#get_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2017B-LP-11/')
#get_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2018A-LP-11/')
#get_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2018B-LP-11/')

#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2020A-LP-104/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2020B-LP-104/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2021A-LP-104/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2021B-LP-104/')
get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2022A-LP-104/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2019B-LP-104/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2020A-LP-104/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2020B-LP-104/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2021A-LP-104/')
#get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2021B-LP-104/')
get_update_object_info('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2022A-LP-104/')

print('Done.')


2022-06-13 12:28:53 - Starting processing data in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2022A-LP-104/...
2022-06-13 12:28:53 - Processing n20220204.104.chip2.otzfc.fits...
2022-06-13 12:28:56 - Processing n20220204.105.chip2.otzfc.fits...
2022-06-13 12:28:59 - Processing n20220204.106.chip2.otzfc.fits...
2022-06-13 12:29:02 - Processing n20220204.107.chip2.otzfc.fits...
2022-06-13 12:29:05 - Processing n20220204.108.chip2.otzfc.fits...
2022-06-13 12:29:07 - Processing n20220204.109.chip2.otzfc.fits...
2022-06-13 12:29:10 - Processing n20220204.110.chip2.otzfc.fits...
2022-06-13 12:29:13 - Processing n20220204.111.chip2.otzfc.fits...
2022-06-13 12:29:16 - Processing n20220209.159.chip2.otzfc.fits...
2022-06-13 12:29:19 - Processing n20220209.160.chip2.otzfc.fits...
2022-06-13 12:29:21 - Processing n20220209.161.chip2.otzfc.fits...
2022-06-13 12:29:24 - Processing n20220209.162.chip2.otzfc.fits...
2022-06-13 12:29:27 - Processing n20220212.077.chip2.otzfc.fits...
2

2022-06-13 12:35:29 - Processing n20220331.101.chip2.otzfc.fits...
2022-06-13 12:35:33 - Processing n20220331.102.chip2.otzfc.fits...
2022-06-13 12:35:37 - Processing n20220331.153.chip2.otzfc.fits...
2022-06-13 12:35:40 - Processing n20220331.154.chip2.otzfc.fits...
2022-06-13 12:35:44 - Processing n20220331.155.chip2.otzfc.fits...
2022-06-13 12:35:48 - Processing n20220401.068.chip2.otzfc.fits...
2022-06-13 12:35:52 - Processing n20220401.069.chip2.otzfc.fits...
2022-06-13 12:35:55 - Processing n20220401.070.chip2.otzfc.fits...
2022-06-13 12:35:59 - Processing n20220423.046.chip2.otzfc.fits...
2022-06-13 12:36:02 - Processing n20220423.047.chip2.otzfc.fits...
2022-06-13 12:36:05 - Processing n20220423.048.chip2.otzfc.fits...
2022-06-13 12:36:09 - Processing n20220526.044.chip2.otzfc.fits...
2022-06-13 12:36:12 - Processing n20220526.045.chip2.otzfc.fits...
2022-06-13 12:36:17 - Processing n20220526.046.chip2.otzfc.fits...
2022-06-13 12:36:21 - Processing n20220526.132.chip2.otzfc.fit